## BioCRNpyler Bioscrape Comparison
### SBML Issue

In this notebook, I simulate the reaction shown below in both bioscrape and biocrnpyler.
<br> <br>
$$\text{2 ATP + Glucose + Enzyme1} \leftrightarrow^1 \text{2ATP:Glucose:Enzyme1} \rightarrow^2 \text{2ADP:F16P:Enzyme1} \leftrightarrow^3 \text{2 ADP + F16P + Enzyme1}$$ <br> <br>


To sum up, the biocrnpyler plots look the same as the bioscrape plots when I use **CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)**. However, the plots looks different when I use **CRN.simulate_with_bioscrape_via_sbml(timepoints, initial_condition_dict = x0, file = 'CRN.sbml')**. For this reason, I think that there may be a small issue with the sbml method.

When the sbml method is used, it seems that the enzyme is used once but is not outputted or reused. The activity of enzyme 1 seems to be the limiting factor. I have closely checked the reactions, rates, and initial conditions and they are identical in both cases.

In [1]:
# Basic Imports

#A Model is a CRN with some bells and whistles
from bioscrape.types import Model

#py_simulate_model is a helper function that takes care of may details for you
from bioscrape.simulator import py_simulate_model

#For arrays and plotting
import numpy as np
#import pylab as plt

# Import good plotting packages 
import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
from bokeh.layouts import column
#import bokeh_catplot
bokeh.io.output_notebook()

import warnings
warnings.filterwarnings("ignore")

from biocrnpyler.mechanism import Mechanism
from biocrnpyler.component import Component
from biocrnpyler import Mixture
from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, ChemicalReactionNetwork
import numpy as np

Loading BokehJS ...

Now, we will make the bioscrape simulation.

# Bioscrape

## Just enzyme 1 reactions

In [2]:
 # 1 step model
species = ['atp', 'glucose', 'e1', '2atp:glucose:e1', '2adp:f16p:e1', 'adp', 'f16p']

# reaction 1
rxn1_f = (['atp', 'atp', 'glucose', 'e1'], ['2atp:glucose:e1'],
           'massaction',  {'k': 'k_bf'})

rxn1_r = ( ['2atp:glucose:e1'],['atp', 'atp', 'glucose', 'e1'], 
            'massaction', {'k': 'k_br'})

# reaction 2
rxn2_f = ( ['2atp:glucose:e1'],['2adp:f16p:e1'], 
           'massaction', {'k': 'k_cat'})

# reaction 3
rxn3_f =  ( ['2adp:f16p:e1'], ['adp', 'adp', 'f16p', 'e1'], 
           'massaction', {'k': 'k_uf'})

rxn3_r =  (['adp', 'adp', 'f16p', 'e1'], ['2adp:f16p:e1'],
           'massaction', {'k': 'k_ur'})

reactions = [rxn1_f, rxn1_r, rxn2_f, rxn3_f, rxn3_r]


# in hrs
e = 0.15
atp = 15
x_0 = {
    "glucose":15,
    "atp":atp, 
    'e1': e,
}
parameters1 = [('k_bf', 22.68), ('k_br', 2.268), ('k_uf', 24), ('k_ur', 2.4), ('k_cat', 10),('k_atp', 0.5)]

M = Model(species = species, reactions = reactions, parameters = parameters1, initial_condition_dict = x_0)
timepoints = np.linspace(0,72,1000)
model_1_df= py_simulate_model(timepoints, Model = M, stochastic = False)

In [3]:
model_1_df

atp    glucose        e1  2atp:glucose:e1  2adp:f16p:e1  \
0    1.500000e+01  15.000000  0.150000     0.000000e+00      0.000000   
1    1.460528e+01  14.802639  0.000247     1.095247e-01      0.040228   
2    1.445788e+01  14.728939  0.000267     1.060213e-01      0.043712   
3    1.430596e+01  14.652982  0.000273     1.057166e-01      0.044011   
4    1.415367e+01  14.576834  0.000277     1.056864e-01      0.044037   
..            ...        ...       ...              ...           ...   
995  1.665478e-16   7.500000  0.012667     3.969753e-17      0.137333   
996  1.663971e-16   7.500000  0.012667     3.966162e-17      0.137333   
997  1.662464e-16   7.500000  0.012667     3.962570e-17      0.137333   
998  1.660958e-16   7.500000  0.012667     3.958978e-17      0.137333   
999  1.659451e-16   7.500000  0.012667     3.955387e-17      0.137333   

           adp      f16p       time  
0     0.000000  0.000000   0.000000  
1     0.095217  0.047608   0.072072  
2     0.242657  0.121328   0.144144  
3     0.394581  0.197291   0.216216  
4     0.546886  0.273443   0.288288  
..         ...       ...        ...  
995  14.725334  7.362667  71.711712  
996  14.725334  7.362667  71.783784  
997  14.725334  7.362667  71.855856  
998  14.725334  7.362667  71.927928  
999  14.725334  7.362667  72.000000  

[1000 rows x 8 columns]

Set up the plotting.

In [4]:
colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']

p1 = bokeh.plotting.figure(width = 450, height = 250, 
                             x_axis_label = 'time',
                             y_axis_label = 'concentration',
                             title = 'All Species with Enzyme1')
p1.line(timepoints, model_1_df['glucose'], color = colors[1],line_width = 2, legend_label = 'glucose')
p1.line(timepoints, model_1_df['f16p'], color = colors[4], line_width = 2,legend_label = 'f16p')
p1.line(timepoints, model_1_df['e1'], color = colors[2], line_width = 2, legend_label = 'e1')
p1.line(timepoints, model_1_df['2atp:glucose:e1'], color = colors[5], line_width = 2, legend_label = 'complex 1')
p1.line(timepoints, model_1_df['2adp:f16p:e1'], color = colors[6], line_width = 2, legend_label = 'complex 2')
p1.legend.location = 'center_right'

# Plot ATP and ADP, Pi
p1.line(timepoints, model_1_df['atp'], color = colors[0], line_width = 2,legend_label = 'atp')
p1.line(timepoints, model_1_df['adp'], color = colors[3], line_width = 2,legend_label = 'adp')

p1.legend.click_policy="hide"

# PLOT JUST THE ENZYME ON ITS OWN PLOT
p2 = bokeh.plotting.figure(width = 450, height = 250, 
                             x_axis_label = 'time',
                             y_axis_label = 'concentration',
                             title = 'Only Enzyme')
p2.line(timepoints, model_1_df['e1'], color = colors[2], line_width = 2, legend_label = 'e1')
p2.legend.location = 'center_right'

bokeh.io.show(row(p1, p2))

Here, on the left hand side, we plot all the species and complexes. Complex 1 is 2atp:glucose:e1 and Complex 2 is 2adp:f16p:e1. On the right hand side, we plot only the enzyme on a small y-axis just as a visualization - it's results are from the same simulation. As we can see, the atp levels go completely to $0$ and f16p, the intermediate, and glucose both reach a steady state level around $7$. We can see that the plot of the enzyme steadies to it's initial condition since it is regenerated.

# Biocrnpyler

Now, we will plot the biocrnpyler results. First, we will define a new mechanism, component, mixture, and auxiliary functions. This auxiliary function below check's the type of a particular item and returns the appropriate version. This code was written as a function for convenience.

## Define Auxiliary Functions

In [5]:
def check_type(item, material_type_str): # call set_species
    if isinstance(item, Species):
        item_ret = item
    elif isinstance(item, str):
        item_ret = Species(name = item, material_type = material_type_str)
    elif isinstance(item, Component) and item.get_species() != None:
        item_ret = item.get_species()
    else:
        raise ValueError( item, "parameter must be a string, a Component with defined get_species(), or a chemical_reaction_network.species")
        
    return item_ret

## Mechanism: FuelMichaelisMenten
This is a mechanism that will be of type 'catalysis'. It will take in an enzyme, fuel_list, substrate_list, product_list, and waste_list. The appropriate complexes, species, and reactions are coded below.

In [6]:
class FuelMichaelisMenten(Mechanism):
    def __init__(self, name, type = 'catalysis', **keywords):
        
        
   
        
        Mechanism.__init__(self = self, name = name, mechanism_type = type, **keywords)

    def update_species(self, enzyme, fuel_list, substrate_list, product_list, waste_list): 
        
        self.enzyme = check_type(enzyme, 'enzyme')
        #self.enzyme = self.set_species(enzyme, material_type = 'enzyme')
        
        species = [self.enzyme]
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]
        
        for f in fuel_list:
            species.append(f)
            comp1_list.append(f)

        for s in substrate_list:
            species.append(s)
            comp1_list.append(s)
            
        for p in product_list:
            species.append(p)
            comp2_list.append(p)
            
        for w in waste_list:
            species.append(w)
            comp2_list.append(w)
               
        
        species += [ComplexSpecies(comp1_list)]
        species += [ComplexSpecies(comp2_list)]
        return species
    
    def update_reactions(self, enzyme, fuel_list, substrate_list, product_list, waste_list, k_cat, component, part_id = None):

        k_bf = 22.68        
        k_br = 2.268
        k_uf = 24
        k_ur = 2.4
       # k_cat = 10
       # k_atp_use = 0.5

        self.enzyme = check_type(enzyme, 'enzyme')
        
        
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]


        for f in fuel_list:
            comp1_list.append(f)
            
        for s in substrate_list:
            comp1_list.append(s)

        for p in product_list:
            comp2_list.append(p)
            
        for w in waste_list:
            comp2_list.append(w)
        
        comp1 = ComplexSpecies(comp1_list)
        comp2 = ComplexSpecies(comp2_list)
      
        
        binding_rxn = Reaction(inputs = comp1_list, outputs=[comp1], k = k_bf, k_rev = k_br)
        cat_rxn = Reaction(inputs = [comp1], outputs = [comp2], k = k_cat)
        unbinding_rxn = Reaction(inputs = [comp2], outputs = comp2_list, k=k_uf, k_rev = k_ur)
        
        return [binding_rxn, cat_rxn, unbinding_rxn]

## Component: Enzyme
Now, we will make a Enzyme component that will take in substrate, fuel, product, and waste. These will be passed into the FuelMichaelisMenten Mechanism.

In [7]:
class Enzyme(Component):
    def __init__(self, enzyme_name, substrate, fuel, product, waste, k_cat = 10, **keywords):
      
        # ENZYME NAME
        # self.enzyme = self.set_species()
        self.enzyme = check_type(enzyme_name, 'enzyme')
    
        # SUBSTRATE
        
        self.substrate_list = []
        for s in substrate:
            self.substrate_list.append(check_type(s, material_type_str ='molecule'))
       
        
        # FUEL
        self.fuel_list = []
        for f in fuel:
            self.fuel_list.append(check_type(f, material_type_str ='metabolite'))
    
        
        # PRODUCT
        self.product_list = []
        for p in product:
            self.product_list.append(check_type(p, 'molecule'))             
            
        # WASTE
        self.waste_list = []
        for w in waste:
            self.waste_list.append(check_type(w, 'metabolite'))
           
        
        self.k_cat = k_cat
            
        
      
        Component.__init__(self = self, name = enzyme_name, **keywords)
        
    def update_species(self):
        mech_cat = self.mechanisms['catalysis']
        
            
        
        return mech_cat.update_species(self.enzyme, self.fuel_list, self.substrate_list, self.product_list, self.waste_list) 
                                                                                           
    
    def update_reactions(self):
        mech_cat = self.mechanisms['catalysis']

        
        return mech_cat.update_reactions(self.enzyme, self.fuel_list, self.substrate_list, self.product_list, self.waste_list,self.k_cat,
                                             component = self, part_id = self.name) # for parameters



## Mixture: EnergyTxTl
Now, we will write a mixture that incorporates the FuelMichaelisMenten mechanism.

In [8]:
class EnergyTxTl(Mixture):
    def __init__(self, name="",**keywords): 
        

        mech_cat = FuelMichaelisMenten('catalysis')
        
        default_mechanisms = {
            mech_cat.mechanism_type:mech_cat
        }
        
        Mixture.__init__(self, name = name, default_mechanisms=default_mechanisms, **keywords) 
    

### Now, we will simulate with only enzyme 1
Biocrnpyler

In [9]:
E1 = Enzyme(enzyme_name = "enzyme1", substrate = ["glucose"], 
            fuel = ['atp', 'atp'],product = ['f16p'], waste = ['adp', 'adp']) #, parameters = parameters)
    

myMixture = EnergyTxTl(components = [E1])
CRN = myMixture.compile_crn()

print(CRN.pretty_print(show_rates = True, show_attributes = False, show_materials = False))

Species (7) = {0. enzyme[enzyme1], 1. metabolite[atp], 2. molecule[glucose], 3. molecule[f16p], 4. metabolite[adp], 5. complex[enzyme[enzyme1]:2x_metabolite[atp]:molecule[glucose]], 6. complex[enzyme[enzyme1]:2x_metabolite[adp]:molecule[f16p]]}
Reactions (3) = [
0. enzyme[enzyme1] + 2 metabolite[atp] + molecule[glucose] <--> complex[enzyme[enzyme1]:2x_metabolite[atp]:molecule[glucose]]        
        massaction: k_f(enzyme[enzyme1],metabolite[atp],molecule[glucose])=22.68*enzyme[enzyme1]*metabolite[atp]^2*molecule[glucose]
        k_r(complex[enzyme[enzyme1]:2x_metabolite[atp]:molecule[glucose]])=2.268*complex[enzyme[enzyme1]:2x_metabolite[atp]:molecule[glucose]]
1. complex[enzyme[enzyme1]:2x_metabolite[atp]:molecule[glucose]] --> complex[enzyme[enzyme1]:2x_metabolite[adp]:molecule[f16p]]        
        massaction: k_f(complex[enzyme[enzyme1]:2x_metabolite[atp]:molecule[glucose]])=10*complex[enzyme[enzyme1]:2x_metabolite[atp]:molecule[glucose]]
2. complex[enzyme[enzyme1]:2x_metabolit

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/component.py:11: UserWarning: get_species() not defined for component {self.name}, None returned.


### With and Without SBML
Here, we will simulate and plot with and without SBML. **re_sbml** is simulating with bioscrape via sbml and **re** is simulating with bioscrape without sbml.

In [10]:
CRN.write_sbml_file("CRN.sbml")
timepoints = np.linspace(0,70,100)
e = 0.15
atp = 15
x0 = {'molecule_glucose':15,
      'metabolite_atp': atp,
      "enzyme_enzyme1":e,}


re_sbml, me= CRN.simulate_with_bioscrape_via_sbml(timepoints, initial_condition_dict = x0, file = 'CRN.sbml')
re = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bioscrape/sbmlutil.py:209: UserWarning: Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. Refer to the bioscrape wiki for more information.
  warnings.warn('Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. ' +


In [11]:
re

enzyme_enzyme1  metabolite_atp  molecule_glucose  molecule_f16p  \
0         0.150000    1.500000e+01         15.000000       0.000000   
1         0.000305    1.326872e+01         14.134358       0.715947   
2         0.000362    1.177575e+01         13.387874       1.462488   
3         0.000438    1.028553e+01         12.642763       2.207675   
4         0.000541    8.800299e+00         11.900149       2.950392   
..             ...             ...               ...            ...   
95        0.012667    4.879477e-16          7.500000       7.362667   
96        0.012667    4.826454e-16          7.500000       7.362667   
97        0.012667    4.773431e-16          7.500000       7.362667   
98        0.012667    4.720408e-16          7.500000       7.362667   
99        0.012667    4.667385e-16          7.500000       7.362667   

    metabolite_adp  complex_enzyme_enzyme1_2x_metabolite_atp_molecule_glucose  \
0         0.000000                                       0.000000e+00           
1         1.431893                                       1.056466e-01           
2         2.924977                                       1.055220e-01           
3         4.415350                                       1.052817e-01           
4         5.900784                                       1.048533e-01           
..             ...                                                ...           
95       14.725334                                       1.162324e-16           
96       14.725334                                       1.149693e-16           
97       14.725334                                       1.137063e-16           
98       14.725334                                       1.124433e-16           
99       14.725334                                       1.111802e-16           

    complex_enzyme_enzyme1_2x_metabolite_adp_molecule_f16p       time  
0                                            0.000000        0.000000  
1                                            0.044049        0.707071  
2                                            0.044116        1.414141  
3                                            0.044280        2.121212  
4                                            0.044605        2.828283  
..                                                ...             ...  
95                                           0.137333       67.171717  
96                                           0.137333       67.878788  
97                                           0.137333       68.585859  
98                                           0.137333       69.292929  
99                                           0.137333       70.000000  

[100 rows x 8 columns]

Now, we will plot the results for **with** and **without** sbml

In [12]:
colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']
# WITHOUT SBML
p1 = bokeh.plotting.figure(width = 450, height = 250, 
                             x_axis_label = 'time',
                             y_axis_label = 'concentration',
                             title = 'WITHOUT SBML')
p1.line(timepoints, re['molecule_glucose'], color = colors[1],line_width = 2, legend_label = 'glucose')
p1.line(timepoints, re['molecule_f16p'], color = colors[4], line_width = 2,legend_label = 'f16p')
p1.line(timepoints, re['enzyme_enzyme1'], color = colors[2], line_width = 2, legend_label = 'e1')
p1.line(timepoints, re['complex_enzyme_enzyme1_2x_metabolite_atp_molecule_glucose'], color = colors[5], line_width = 2, legend_label = 'complex 1')
p1.line(timepoints, re['complex_enzyme_enzyme1_2x_metabolite_adp_molecule_f16p'], color = colors[6], line_width = 2, legend_label = 'complex 2')
p1.legend.location = 'center_right'

# Plot ATP and ADP, Pi
p1.line(timepoints, re['metabolite_atp'], color = colors[0], line_width = 2,legend_label = 'atp')
p1.line(timepoints, re['metabolite_adp'], color = colors[3], line_width = 2,legend_label = 'adp')

p1.legend.click_policy="hide"

# plot just enzyme
p2 = bokeh.plotting.figure(width = 450, height = 250, 
                             x_axis_label = 'time',
                             y_axis_label = 'concentration',
                             title = 'Only Enzyme without SBML')
p2.line(timepoints, re['enzyme_enzyme1'], color = colors[2], line_width = 2, legend_label = 'e1')
p2.legend.location = 'center_right'


# WITH SBML
p3 = bokeh.plotting.figure(width = 450, height = 250, 
                             x_axis_label = 'time',
                             y_axis_label = 'concentration',
                             title = 'WITH SBML')
p3.line(timepoints, re_sbml['molecule_glucose'], color = colors[1],line_width = 2, legend_label = 'glucose')
p3.line(timepoints, re_sbml['molecule_f16p'], color = colors[4], line_width = 2,legend_label = 'f16p')
p3.line(timepoints, re_sbml['enzyme_enzyme1'], color = colors[2], line_width = 2, legend_label = 'e1')
p3.line(timepoints, re_sbml['complex_enzyme_enzyme1_2x_metabolite_atp_molecule_glucose'], color = colors[5], line_width = 2, legend_label = 'complex 1')
p3.line(timepoints, re_sbml['complex_enzyme_enzyme1_2x_metabolite_adp_molecule_f16p'], color = colors[6], line_width = 2, legend_label = 'complex 2')
p3.legend.location = 'center_right'

# plot just enzyme
p4 = bokeh.plotting.figure(width = 450, height = 250, 
                             x_axis_label = 'time',
                             y_axis_label = 'concentration',
                             title = 'Only Enzyme with SBML')
p4.line(timepoints, re_sbml['enzyme_enzyme1'], color = colors[2], line_width = 2, legend_label = 'e1')
p4.legend.location = 'center_right'



bokeh.io.show(row(p1, p2))
bokeh.io.show(row(p3, p4))

As we can see here, there is a difference between the simulation with or without SBML. Without SBML, we see the same results as done by bioscrape. We see that there is regeneration of enzyme without SBML but no regeneration with SBML. The glucose is not used with SBML. This seems to be a bug with the code perhaps? 

Please let me know if there's anything else I can do to help!

In [13]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,jupyterlab,bioscrape,biocrnpyler

CPython 3.7.6
IPython 7.12.0

numpy 1.18.1
bokeh 2.0.2
jupyterlab 1.2.6
bioscrape 0.0.0
biocrnpyler unknown
